In [3]:
import pystan as stan
from scipy.special import logit, expit
from numpy.random import normal, choice, binomial, uniform
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#ckmr_model = stan.CmdStanModel(stan_file='~/src/ckmr/src/sib.stan')
ckmr_model_full = stan.StanModel(file='/home/alex/src/ckmr/src/sib_full.stan')

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_2ac5f2fa35f98226983048f9ffff264b NOW.
In file included from /nix/store/w0nkg5cx8d3lf880gxpi1zbjgjw0a84c-python3-3.8.12-env/lib/python3.8/site-packages/numpy/core/include/numpy/ndarraytypes.h:1969,
                 from /nix/store/w0nkg5cx8d3lf880gxpi1zbjgjw0a84c-python3-3.8.12-env/lib/python3.8/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /nix/store/w0nkg5cx8d3lf880gxpi1zbjgjw0a84c-python3-3.8.12-env/lib/python3.8/site-packages/numpy/core/include/numpy/arrayobject.h:4,
                 from /run/user/1000/pystan_lt3f41_k/stanfit4anon_model_2ac5f2fa35f98226983048f9ffff264b_1020999159275558894.cpp:709:
/nix/store/w0nkg5cx8d3lf880gxpi1zbjgjw0a84c-python3-3.8.12-env/lib/python3.8/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using depr

/run/user/1000/pystan_lt3f41_k/stanfit4anon_model_2ac5f2fa35f98226983048f9ffff264b_1020999159275558894.cpp: In function ‘PyObject* __pyx_pf_71stanfit4anon_model_2ac5f2fa35f98226983048f9ffff264b_1020999159275558894_2_call_sampler(PyObject*, PyObject*, PyObject*, PyObject*)’:
/run/user/1000/pystan_lt3f41_k/stanfit4anon_model_2ac5f2fa35f98226983048f9ffff264b_1020999159275558894.cpp:9482:30: warning: comparison of integer expressions of different signedness: ‘Py_ssize_t’ {aka ‘long int’} and ‘std::vector<std::__cxx11::basic_string<char> >::size_type’ {aka ‘long unsigned int’} [-Wsign-compare]
 9482 |     __pyx_t_12 = ((__pyx_t_9 != __pyx_v_fitptr->param_names_oi().size()) != 0);
      |                    ~~~~~~~~~~^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
In file included from /nix/store/cq152id4fljawrl13kajy9mq89ych5gf-python3.8-pystan-2.19.1.1/lib/python3.8/site-packages/pystan/stan/lib/stan_math/lib/eigen_3.3.3/Eigen/Core:420,
                 from /nix/store/cq152id4fljawrl13kajy9mq8

KeyboardInterrupt: 

In [ ]:
df = pd.read_excel('PIRU_mykiss_parentage.xlsx')
df = df[df['Count'] > 0]
df = df[['Count']]

In [ ]:
def preprocess_data(df):
    df = df[['OffspringID', 'MotherID', 'FatherID']]
    df = df.groupby(['MotherID', 'FatherID'], as_index=False).count()
    df = df.rename(columns={'OffspringID': 'Count'})
    df = df[df['Count'] > 0]
    return df
    
def sub_sample(df, p=0.1):
    tot = df['Count'].sum()
    new_count = binomial(df['Count'], p)
    df1 = df[set(df.columns).difference({'Count'})]
    df1['Count'] = new_count
    df1 = df1[df1['Count'] > 0]
    return df1
    

def make_stan_data(df, method='normal', osr=[0.0, 0.01], prom=[-2.5, 0.11]):
    if method == 'normal':
        method = 1
    elif method == 'beta':
        method = 2
    else:
        raise ValueError(f'Invalid method: {method}')
    data = {
        'method': method,
        'osr_mu': osr[0],
        'osr_sigma': osr[1],
        'G': len(df['Count']),
        'M': df['FatherID'].nunique(),
        'F': df['MotherID'].nunique(),
        'N': df['Count'].sum(),
        'grp_size': df['Count'].values
    }
    return data

def plot_results(dfs, field='F0', ci=0.99, title='', label='', ax=None):
    dfs = summarize_samples(dfs, ci=ci, field=field)
    ns = [d[0] for d in dfs]
    summaries = [d[1] for d in dfs]
    if ax is None:
        _, ax = plt.subplots(figsize=(16, 9))
    y0 = [s[field].loc['mean'] for s in summaries]
    y1 = [s[field].iloc[4] for s in summaries]
    y2 = [s[field].iloc[6] for s in summaries]
    ax.plot(ns, y0, 'o-', label=f'{label} mean')
    ax.fill_between(ns, y1, y2, alpha=0.5, label=f'{label} {(100*ci):.1f}% CI')
    ax.set_xlabel('sample size')
    ax.set_ylabel(f'{field} estimate')
    ax.set_title(title)
    return ax
    
def sample_data(dfs, method='normal', osr=[0.0, 0.01], **kwargs):
    fits = [(d['Count'].sum(), ckmr_model_full.sampling(data=make_stan_data(d, method=method, osr=osr), **kwargs)) for d in dfs]
    fits.sort(key=(lambda x: x[0]))
    return fits

def summarize_samples(fits, ci=0.99, field='F0'):
    lower = 0.5 * (1.0 - ci)
    upper = 1.0 - lower
    quants = (lower, upper)
    res = [(n, d.to_dataframe(pars=[field]).describe(percentiles=quants)) for (n, d) in fits]
    return res
    
def make_synth_data(M0, F0, G0, N):
    N0 = M0 + F0
    gs = choice(M0*F0, size=G0, replace=False)
    gs = np.array([[g // F0, g % F0] for g in gs], dtype=int).T
    xs = choice(G0, size=N)
    df = pd.DataFrame(data={'OffspringID': np.arange(N), 'FatherID': gs[0, xs], 'MotherID': gs[1, xs]})
    return df

In [ ]:
dfs = (sub_sample(df, p=p) for p in np.arange(0.05, 1.0, 0.05))
fits = sample_data(dfs, chains=4, show_progress=False, iter_sampling=2000)

In [ ]:
plot_results(fits, field='F0')

In [ ]:
plot_results(fits, field='N0')

In [ ]:
dfs1 = (make_synth_data(137, N) for N in range(100, 1000, 100))
fits1 = sample_data(dfs1, chains=4, show_progress=False, iter_sampling=2000)

In [ ]:
plot_results(fits1, field='F0')

In [ ]:
plot_results(fits1, field='N0')

In [ ]:
dfs2 = (make_synth_data(1234, N) for N in range(200, 2200, 200))
fits2 = sample_data(dfs2, chains=4, show_progress=False, iter_sampling=2000)

In [ ]:
plot_results(fits2, field='F0')

In [ ]:
plot_results(fits2, field='N0')

In [ ]:
df3 = preprocess_data(pd.read_csv('/home/alex/src/ckmr/Stan_mykiss_2021_Run1.BestConfig'))

In [ ]:
df3

In [ ]:
dfs3 = [sub_sample(df3, p=p) for p in np.linspace(0.2, 1.0, 18)]
norm3 = sample_data(dfs3, method='normal', chains=4, iter=2000)
beta3 = sample_data(dfs3, method='beta', chains=4, iter=2000)

In [ ]:
ax = plot_results(norm3[1:], field='G0', title='Estimated number of full sibling groups')
ax = plot_results(beta3[1:], field='G0', title='Estimated number of full sibling groups', ax=ax)
ax.legend()

In [ ]:
ax = plot_results(norm3[1:], field='N0', title='Estimated number of adults')
ax = plot_results(beta3[1:], field='N0', title='Estimated number of adults', ax=ax)
ax.legend()

In [ ]:
norm3[-1][1].to_dataframe(pars=['G0', 'N0', 'M0', 'F0', 'osr']).describe(percentiles=[0.005, 0.995])[['G0', 'N0', 'M0', 'F0', 'osr']]

In [ ]:
beta3[-1][1].to_dataframe(pars=['G0', 'N0', 'M0', 'F0', 'osr']).describe(percentiles=[0.005, 0.995])[['G0', 'N0', 'M0', 'F0', 'osr']]

In [ ]:
print(beta3[-1][1].stansummary())

In [ ]:
plt.scatter(norm3[-1][1].draws_pd(['F0'])['F0'], norm3[-1][1].draws_pd(['M0'])['M0'], alpha=0.3, label='normal')
plt.scatter(beta3[-1][1].draws_pd(['F0'])['F0'], beta3[-1][1].draws_pd(['M0'])['M0'], alpha=0.3, label='beta')
plt.xlabel('females')
plt.ylabel('males')
plt.legend()

In [ ]:
beta3[-1][1].diagnose()

In [ ]:
df4 = preprocess_data(pd.read_csv('/home/alex/src/ckmr/Piru_mykiss-Run1.BestConfig.txt'))

In [ ]:
dfs4 = [sub_sample(df4, p=p) for p in np.linspace(0.1, 1.0, 19)]
res4 = sample_data(dfs4, osr=[0.0, 0.01], chains=4, iter=2000)
#beta4 = sample_data(dfs4, method='beta', osr=[0.0, 10.0], chains=4, iter=2000)

In [ ]:
ax = plot_results(res4, field='G0', title='Estimated number of full sibling groups')
#ax = plot_results(beta4, field='G0', title='Estimated number of full sibling groups', ax=ax)
ax.legend()

In [ ]:
ax = plot_results(res4, field='N0', title='Estimated number of adults')
#ax = plot_results(beta4, field='N0', title='Estimated number of adults', ax=ax)
ax.legend()

In [ ]:
res4[-1][1].to_dataframe(['G0', 'N0', 'M0', 'F0', 'osr']).describe(percentiles=[0.005, 0.995])[['G0', 'N0', 'M0', 'F0', 'osr']]

In [ ]:
stan.check_hmc_diagnostics(res4[-1][1])

In [ ]:
df5 = preprocess_data(make_synth_data(12149, 12152, 155743, 2000))
dfs5 = [sub_sample(df5, p=p) for p in np.linspace(0.05, 1.0, 20)]
res5 = sample_data(dfs5, osr=[0.0, 0.1], chains=4, iter=2000)
#beta5 = sample_data(dfs5, method='beta', osr=[0.0, 0.1], chains=4, iter=2000)

In [ ]:
ax = plot_results(res5[5:], field='G0', title='Estimated number of full sibling groups', label='unified')
#ax = plot_results(beta5[2:], field='G0', title='Estimated number of full sibling groups', label='beta', ax=ax)
ax.legend()

In [ ]:
ax = plot_results(res5[5:], field='N0', title='Estimated number of adults', label='unified')
#ax = plot_results(beta5[2:], field='N0', title='Estimated number of adults', label='beta', ax=ax)
ax.legend()

In [ ]:
res5[-1][1].to_dataframe(pars=['G0', 'N0', 'M0', 'F0', 'osr']).describe(percentiles=[0.005, 0.995])[['G0', 'N0', 'M0', 'F0', 'osr']]

In [ ]:
df5

In [ ]:
df5.nunique()

In [49]:
stan.check_hmc_diagnostics(res5[-1][1])

{'n_eff': True,
 'Rhat': True,
 'divergence': True,
 'treedepth': True,
 'energy': True}

In [5]:
piru = pd.read_excel('PIRU_mykiss_parentage.xlsx')

In [6]:
piru

,FullSibshipIndex,Prob(Inc.),Prob(Exc.),Member1,Member2,Member3,Member4,Member5,Member6,Member7,...,Member58,Member59,Member60,Member61,Member62,Member63,Member64,FullSibshipIndex.1,Unnamed: 68,Count
0,1,1.0,0.5498,989001028888212,989001028888256,989001028888041,PIRU43,989001028888249,989001028888234,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,7
1,2,1.0,0.5499,PIRU04,PIRU16,989001028888264,PIRU30,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,5
2,3,1.0,1.0000,989001028888228,989001028888175,PIRU31,989001028888207,PIRU68,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,11
3,5,1.0,0.4347,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,0
4,6,1.0,1.0000,989001028888173,989001028888189,989001028888213,PIRU1017,989001028888089,989001028888088,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,NaN,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,194,1.0,0.2587,861326,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,194,NaN,1
134,195,1.0,0.4325,861372,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,195,NaN,1
135,197,1.0,0.3916,989001028888246,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,197,NaN,1
136,199,1.0,0.9995,989001028888244,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,199,NaN,1
